# 📡 Uso de APIs LLM - OpenAI e Anthropic
 
## Este notebook ensina como usar as principais APIs de LLM para análise de código.
 
### ## Objetivos:
### 1. Configurar APIs OpenAI e Anthropic
### 2. Comparar diferentes modelos
### 3. Entender parâmetros importantes
### 4. Criar um comparador para análise de código

### 1. Instalação e Configuração

In [ ]:
# Instalar bibliotecas necessárias
#!pip install openai anthropic python-dotenv tiktoken
#!pip uninstall anthropic

In [1]:
# Instalar bibliotecas necessárias
!pip install openai python-dotenv tiktoken ollama

### 🔑 2. Configuração via Arquivo .env

In [2]:
import os
from dotenv import load_dotenv
import subprocess
import sys

# Carregar variáveis do arquivo .env
load_dotenv()

# Verificar e configurar .env se necessário
def setup_environment():
    """Configura o ambiente e cria arquivo .env se não existir"""
    
    env_file = '.env'
    
    if not os.path.exists(env_file):
        print("📝 Criando arquivo .env...")
        
        # Pedir OpenAI API Key ao usuário
        openai_key = input("Digite sua OpenAI API Key (ou Enter para pular): ").strip()
        
        # Criar conteúdo do .env
        env_content = f"""# OpenAI API Key (obtenha em: https://platform.openai.com/api-keys)
# Começa com 'sk-'
OPENAI_API_KEY={openai_key if openai_key else 'sua-chave-openai-aqui'}

# Configurações do Ollama (local)
OLLAMA_MODEL=llama3
OLLAMA_BASE_URL=http://localhost:11434

# Configurações da aplicação
TEMPERATURE_CODE_ANALYSIS=0.1
MAX_TOKENS_CODE_ANALYSIS=500
"""
        
        with open(env_file, 'w', encoding='utf-8') as f:
            f.write(env_content)
        
        print(f"✅ Arquivo {env_file} criado com sucesso!")
        print("📋 Você pode editá-lo manualmente para atualizar as configurações.")
    
    # Recarregar variáveis
    load_dotenv(override=True)
    
    return env_file

# Executar setup
env_file = setup_environment()

# Verificar configurações
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama3")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")

print("\n" + "="*50)
print("🔍 CONFIGURAÇÃO ATUAL:")
print("="*50)
print(f"✅ Arquivo .env: {env_file}")
print(f"🔑 OpenAI API Key: {'✅ Configurada' if OPENAI_API_KEY and OPENAI_API_KEY != 'sua-chave-openai-aqui' else '❌ Não configurada'}")
print(f"🤖 Ollama Model: {OLLAMA_MODEL}")
print(f"🌐 Ollama URL: {OLLAMA_BASE_URL}")
print("="*50)


🔍 CONFIGURAÇÃO ATUAL:
✅ Arquivo .env: .env
🔑 OpenAI API Key: ✅ Configurada
🤖 Ollama Model: llama3
🌐 Ollama URL: http://localhost:11434


### 🤖 3. Configurar Ollama (Local)

In [3]:
def setup_ollama():
    """Configura e verifica o Ollama local"""
    
    print("\n" + "="*50)
    print("🔄 CONFIGURANDO OLLAMA (LOCAL)")
    print("="*50)
    
    try:
        # Verificar se Ollama está instalado
        result = subprocess.run(['ollama', '--version'], 
                              capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"✅ Ollama encontrado: {result.stdout.strip()}")
        else:
            print("❌ Ollama não encontrado. Tentando instalar...")
            
            # Instalar Ollama (Linux/Mac)
            if sys.platform != 'win32':
                !curl -fsSL https://ollama.com/install.sh | sh
            else:
                print("⚠️  Para Windows, baixe em: https://ollama.com/download")
                print("⚠️  Execute o instalador e reinicie o terminal.")
                return False
            
            # Verificar novamente
            result = subprocess.run(['ollama', '--version'], 
                                  capture_output=True, text=True)
            if result.returncode != 0:
                print("❌ Falha na instalação do Ollama")
                return False
    
    except FileNotFoundError:
        print("❌ Ollama não está instalado.")
        install = input("Deseja instalar o Ollama agora? (s/n): ")
        
        if install.lower() == 's':
            if sys.platform != 'win32':
                !curl -fsSL https://ollama.com/install.sh | sh
            else:
                print("⚠️  Para Windows, baixe manualmente em: https://ollama.com/download")
                return False
        else:
            print("⚠️  Ollama não será usado. Continuando apenas com OpenAI.")
            return False
    
    # Verificar/puxar modelo
    try:
        print(f"\n🔍 Verificando modelo '{OLLAMA_MODEL}'...")
        result = subprocess.run(['ollama', 'list'], 
                              capture_output=True, text=True)
        
        if OLLAMA_MODEL in result.stdout:
            print(f"✅ Modelo '{OLLAMA_MODEL}' já está baixado.")
        else:
            print(f"⬇️  Baixando modelo '{OLLAMA_MODEL}'...")
            !ollama pull {OLLAMA_MODEL}
        
        return True
        
    except Exception as e:
        print(f"❌ Erro ao configurar Ollama: {str(e)}")
        return False

# Executar setup do Ollama
ollama_ready = setup_ollama()


🔄 CONFIGURANDO OLLAMA (LOCAL)
❌ Ollama não está instalado.


Deseja instalar o Ollama agora? (s/n):  s


⚠️  Para Windows, baixe manualmente em: https://ollama.com/download


### ⚙️ 4. Inicializar Clientes

In [ ]:
from openai import OpenAI
import ollama

# Configurar cliente OpenAI
openai_client = None
if OPENAI_API_KEY and OPENAI_API_KEY != 'sua-chave-openai-aqui':
    try:
        openai_client = OpenAI(api_key=OPENAI_API_KEY)
        print("✅ Cliente OpenAI configurado com sucesso!")
    except Exception as e:
        print(f"❌ Erro ao configurar OpenAI: {str(e)}")
else:
    print("⚠️  OpenAI não configurado. Usando apenas Ollama local.")

# Configurar Ollama
if ollama_ready:
    try:
        # Testar conexão com Ollama
        response = ollama.chat(model=OLLAMA_MODEL, messages=[
            {'role': 'user', 'content': 'Hello'}
        ])
        print(f"✅ Ollama conectado! Modelo: {OLLAMA_MODEL}")
    except Exception as e:
        print(f"❌ Erro ao conectar com Ollama: {str(e)}")
        ollama_ready = False
else:
    print("⚠️  Ollama não disponível.")

print("\n" + "="*50)
print("📊 STATUS DOS CLIENTES:")
print("="*50)
print(f"OpenAI: {'✅ Pronto' if openai_client else '❌ Não disponível'}")
print(f"Ollama: {'✅ Pronto' if ollama_ready else '❌ Não disponível'}")
print("="*50)

if not openai_client and not ollama_ready:
    print("\n⚠️  ATENÇÃO: Nenhum cliente LLM configurado!")
    print("Configure pelo menos um dos dois para continuar.")

### 🧪 5. Funções Básicas de Análise

In [ ]:
def analyze_with_openai(code, question, model="gpt-3.5-turbo"):
    """Analisa código usando OpenAI"""
    
    if not openai_client:
        return "❌ OpenAI não configurado"
    
    prompt = f"""
    Você é um especialista em análise de código Python.
    
    Por favor, analise este código:
    
    ```python
    {code}
    ```
    
    Responda à pergunta: {question}
    
    Forneça uma análise técnica detalhada.
    """
    
    try:
        response = openai_client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Você é um analista de código especializado."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=int(os.getenv("MAX_TOKENS_CODE_ANALYSIS", 500)),
            temperature=float(os.getenv("TEMPERATURE_CODE_ANALYSIS", 0.1))
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        return f"❌ Erro OpenAI: {str(e)}"

def analyze_with_ollama(code, question):
    """Analisa código usando Ollama local"""
    
    if not ollama_ready:
        return "❌ Ollama não configurado"
    
    prompt = f"""
    Você é um especialista em análise de código Python.
    
    Por favor, analise este código:
    
    ```python
    {code}
    ```
    
    Responda à pergunta: {question}
    
    Forneça uma análise técnica detalhada em português.
    """
    
    try:
        response = ollama.chat(
            model=OLLAMA_MODEL,
            messages=[
                {'role': 'system', 'content': 'Você é um analista de código especializado.'},
                {'role': 'user', 'content': prompt}
            ],
            options={
                'temperature': float(os.getenv("TEMPERATURE_CODE_ANALYSIS", 0.1)),
                'num_predict': int(os.getenv("MAX_TOKENS_CODE_ANALYSIS", 500))
            }
        )
        
        return response['message']['content']
        
    except Exception as e:
        return f"❌ Erro Ollama: {str(e)}"

### 🔍 6. Teste Prático: Analisar Código Real

In [ ]:
# Código exemplo do seu repositório ESPECIALISTA_EM_AI
sample_codes = {
    "CNN simples": """
import tensorflow as tf
from tensorflow.keras import layers

def create_simple_cnn(input_shape=(32, 32, 3), num_classes=10):
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model
""",
    
    "Busca binária": """
def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    
    while left <= right:
        mid = (left + right) // 2
        
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    
    return -1
""",
    
    "Processamento de texto": """
import re
from collections import Counter

def analyze_text(text):
    # Limpar texto
    text = text.lower()
    text = re.sub(r'[^a-z\\s]', '', text)
    
    # Contar palavras
    words = text.split()
    word_count = len(words)
    unique_words = len(set(words))
    
    # Palavras mais comuns
    common_words = Counter(words).most_common(5)
    
    return {
        'total_words': word_count,
        'unique_words': unique_words,
        'common_words': common_words
    }
"""
}

# Perguntas para teste
test_questions = [
    "Esta é uma boa implementação? Quais melhorias você sugere?",
    "Qual é a complexidade deste algoritmo?",
    "Existe algum bug potencial neste código?"
]

### 📊 7. Executar Análises Comparativas

In [ ]:
import time
import pandas as pd
from IPython.display import display, HTML

def run_comparative_analysis():
    """Executa análise comparativa entre OpenAI e Ollama"""
    
    results = []
    
    for code_name, code in sample_codes.items():
        print(f"\n{'='*60}")
        print(f"🔍 ANALISANDO: {code_name}")
        print(f"{'='*60}")
        
        for question in test_questions:
            print(f"\n❓ Pergunta: {question}")
            print(f"{'-'*40}")
            
            # Analisar com OpenAI (se disponível)
            if openai_client:
                print("🤖 OpenAI GPT-3.5-Turbo:")
                start_time = time.time()
                openai_result = analyze_with_openai(code, question)
                openai_time = time.time() - start_time
                
                # Mostrar resumo
                print(f"   ⏱️  Tempo: {openai_time:.2f}s")
                print(f"   📝 Resumo: {openai_result[:100]}...")
                
                results.append({
                    'Código': code_name,
                    'Pergunta': question[:30] + "...",
                    'Modelo': 'OpenAI GPT-3.5',
                    'Tempo (s)': round(openai_time, 2),
                    'Comprimento': len(openai_result),
                    'Resposta': openai_result[:150] + "..."
                })
            
            # Analisar com Ollama (se disponível)
            if ollama_ready:
                print("\n🖥️  Ollama (Local):")
                start_time = time.time()
                ollama_result = analyze_with_ollama(code, question)
                ollama_time = time.time() - start_time
                
                # Mostrar resumo
                print(f"   ⏱️  Tempo: {ollama_time:.2f}s")
                print(f"   📝 Resumo: {ollama_result[:100]}...")
                
                results.append({
                    'Código': code_name,
                    'Pergunta': question[:30] + "...",
                    'Modelo': f'Ollama {OLLAMA_MODEL}',
                    'Tempo (s)': round(ollama_time, 2),
                    'Comprimento': len(ollama_result),
                    'Resposta': ollama_result[:150] + "..."
                })
            
            print(f"{'-'*40}")
    
    # Criar DataFrame com resultados
    if results:
        df = pd.DataFrame(results)
        
        print(f"\n{'='*60}")
        print("📊 RESUMO COMPARATIVO")
        print(f"{'='*60}")
        
        # Agrupar por modelo para estatísticas
        if len(df) > 0:
            stats = df.groupby('Modelo').agg({
                'Tempo (s)': ['mean', 'min', 'max'],
                'Comprimento': 'mean'
            }).round(2)
            
            display(stats)
        
        return df
    else:
        print("⚠️  Nenhum resultado para mostrar.")
        return None

# Executar análises
comparison_df = run_comparative_analysis()

### ⚡ 8. Streaming em Tempo Real

In [ ]:
def stream_analysis(code, question, provider="ollama"):
    """Mostra análise com streaming em tempo real"""
    
    prompt = f"""
    Analise este código Python:
    
    ```python
    {code}
    ```
    
    Pergunta: {question}
    
    Responda de forma clara e técnica.
    """
    
    print(f"\n🎯 Analisando com {provider.upper()}...")
    print("="*50)
    
    if provider == "openai" and openai_client:
        try:
            stream = openai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=300,
                temperature=0.1,
                stream=True
            )
            
            for chunk in stream:
                if chunk.choices[0].delta.content is not None:
                    print(chunk.choices[0].delta.content, end="", flush=True)
                    
        except Exception as e:
            print(f"\n❌ Erro: {str(e)}")
    
    elif provider == "ollama" and ollama_ready:
        try:
            # Ollama também suporta streaming
            response = ollama.generate(
                model=OLLAMA_MODEL,
                prompt=prompt,
                stream=True
            )
            
            for chunk in response:
                if 'response' in chunk:
                    print(chunk['response'], end="", flush=True)
                    
        except Exception as e:
            print(f"\n❌ Erro: {str(e)}")
    
    else:
        print(f"❌ Provedor '{provider}' não disponível.")
    
    print("\n" + "="*50)

# Testar streaming (escolha um provedor)
if openai_client or ollama_ready:
    # Usar primeiro provedor disponível
    provider = "openai" if openai_client else "ollama"
    stream_analysis(sample_codes["Busca binária"], 
                   "Esta implementação está correta?", 
                   provider=provider)

### 📈 9. Benchmark de Performance

In [ ]:
def run_benchmark(iterations=2):
    """Executa benchmark de performance"""
    
    benchmark_results = []
    code_to_test = sample_codes["Busca binária"]
    question = "Qual é a complexidade deste algoritmo?"
    
    print(f"\n{'='*60}")
    print(f"🏃‍♂️  BENCHMARK ({iterations} iterações)")
    print(f"{'='*60}")
    
    # Testar OpenAI
    if openai_client:
        print("\n🤖 Testando OpenAI GPT-3.5-Turbo...")
        openai_times = []
        
        for i in range(iterations):
            start_time = time.time()
            result = analyze_with_openai(code_to_test, question)
            elapsed = time.time() - start_time
            openai_times.append(elapsed)
            
            print(f"  Iteração {i+1}: {elapsed:.2f}s ({len(result)} chars)")
        
        avg_time = sum(openai_times) / len(openai_times)
        benchmark_results.append({
            'Modelo': 'OpenAI GPT-3.5',
            'Tempo Médio (s)': round(avg_time, 2),
            'Iterações': iterations,
            'Tipo': 'Cloud'
        })
    
    # Testar Ollama
    if ollama_ready:
        print(f"\n🖥️  Testando Ollama {OLLAMA_MODEL}...")
        ollama_times = []
        
        for i in range(iterations):
            start_time = time.time()
            result = analyze_with_ollama(code_to_test, question)
            elapsed = time.time() - start_time
            ollama_times.append(elapsed)
            
            print(f"  Iteração {i+1}: {elapsed:.2f}s ({len(result)} chars)")
        
        avg_time = sum(ollama_times) / len(ollama_times)
        benchmark_results.append({
            'Modelo': f'Ollama {OLLAMA_MODEL}',
            'Tempo Médio (s)': round(avg_time, 2),
            'Iterações': iterations,
            'Tipo': 'Local'
        })
    
    # Mostrar resultados
    if benchmark_results:
        benchmark_df = pd.DataFrame(benchmark_results)
        
        print(f"\n{'='*60}")
        print("📊 RESULTADOS DO BENCHMARK")
        print(f"{'='*60}")
        
        display(benchmark_df)
        
        # Análise
        print("\n💡 ANÁLISE:")
        for _, row in benchmark_df.iterrows():
            print(f"  • {row['Modelo']}: {row['Tempo Médio (s)']}s ({row['Tipo']})")
        
        return benchmark_df
    else:
        print("⚠️  Nenhum modelo disponível para benchmark.")
        return None

# Executar benchmark
benchmark_df = run_benchmark(iterations=2)

### 🛠️ 10. Sistema de Análise Inteligente

In [ ]:
class CodeAnalysisAssistant:
    """Assistente inteligente para análise de código"""
    
    def __init__(self, use_openai=True, use_ollama=True):
        self.use_openai = use_openai and openai_client is not None
        self.use_ollama = use_ollama and ollama_ready
        
        print(f"🤖 Assistente inicializado:")
        print(f"   • OpenAI: {'✅' if self.use_openai else '❌'}")
        print(f"   • Ollama: {'✅' if self.use_ollama else '❌'}")
    
    def smart_analyze(self, code, question):
        """Escolhe o melhor modelo para análise"""
        
        # Heurística simples: usa Ollama para códigos curtos, OpenAI para complexos
        code_complexity = self._estimate_complexity(code)
        
        if code_complexity == "alta" and self.use_openai:
            print("🔍 Usando OpenAI (código complexo)...")
            return analyze_with_openai(code, question)
        elif self.use_ollama:
            print("🔍 Usando Ollama (código simples/local)...")
            return analyze_with_ollama(code, question)
        elif self.use_openai:
            print("🔍 Usando OpenAI (fallback)...")
            return analyze_with_openai(code, question)
        else:
            return "❌ Nenhum modelo disponível para análise."
    
    def _estimate_complexity(self, code):
        """Estima complexidade do código"""
        lines = code.strip().split('\n')
        
        if len(lines) < 10:
            return "baixa"
        elif len(lines) < 30:
            return "média"
        else:
            return "alta"
    
    def compare_analyses(self, code, question):
        """Compara análises de diferentes modelos"""
        
        analyses = {}
        
        if self.use_openai:
            print("\n🤖 Obtendo análise da OpenAI...")
            analyses['OpenAI'] = analyze_with_openai(code, question)
        
        if self.use_ollama:
            print("🖥️  Obtendo análise do Ollama...")
            analyses['Ollama'] = analyze_with_ollama(code, question)
        
        # Mostrar comparação
        print(f"\n{'='*60}")
        print("🆚 COMPARAÇÃO DE ANÁLISES")
        print(f"{'='*60}")
        
        for model, analysis in analyses.items():
            print(f"\n📋 {model}:")
            print(f"{'─'*40}")
            print(analysis[:300] + "..." if len(analysis) > 300 else analysis)
        
        return analyses

# Testar o assistente
if openai_client or ollama_ready:
    print("\n" + "="*60)
    print("🧪 TESTANDO ASSISTENTE INTELIGENTE")
    print("="*60)
    
    assistant = CodeAnalysisAssistant()
    
    # Testar análise inteligente
    test_code = sample_codes["Processamento de texto"]
    test_question = "Esta função é eficiente para processar grandes textos?"
    
    print(f"\n❓ Pergunta: {test_question}")
    result = assistant.smart_analyze(test_code, test_question)
    print(f"\n📝 Resposta: {result[:200]}...")
    
    # Testar comparação
    if assistant.use_openai and assistant.use_ollama:
        print(f"\n{'='*60}")
        print("🔄 COMPARAÇÃO DETALHADA")
        print("="*60)
        assistant.compare_analyses(test_code, test_question)

### 💾 11. Salvar Resultados e Configurações

In [ ]:
import json
from datetime import datetime

def save_analysis_session(results, filename="analysis_session.json"):
    """Salva toda a sessão de análise para referência futura"""
    
    session_data = {
        "timestamp": datetime.now().isoformat(),
        "config": {
            "openai_available": openai_client is not None,
            "ollama_available": ollama_ready,
            "ollama_model": OLLAMA_MODEL,
            "temperature": os.getenv("TEMPERATURE_CODE_ANALYSIS"),
            "max_tokens": os.getenv("MAX_TOKENS_CODE_ANALYSIS")
        },
        "sample_codes": {k: v[:100] + "..." for k, v in sample_codes.items()},
        "results": results
    }
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(session_data, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Sessão salva em: {filename}")
    
    # Também salvar um resumo em Markdown
    summary_file = "analysis_summary.md"
    with open(summary_file, 'w', encoding='utf-8') as f:
        f.write(f"# Resumo da Análise de Código\n\n")
        f.write(f"**Data**: {datetime.now().strftime('%d/%m/%Y %H:%M')}\n\n")
        
        f.write("## Configuração\n")
        f.write(f"- OpenAI: {'✅' if openai_client else '❌'}\n")
        f.write(f"- Ollama: {'✅' if ollama_ready else '❌'} ({OLLAMA_MODEL})\n\n")
        
        if comparison_df is not None:
            f.write("## Resultados Comparativos\n")
            f.write(comparison_df.to_markdown(index=False))
            f.write("\n\n")
        
        if benchmark_df is not None:
            f.write("## Benchmark de Performance\n")
            f.write(benchmark_df.to_markdown(index=False))
    
    print(f"📝 Resumo em Markdown: {summary_file}")
    
    return session_data

# Salvar resultados se houver
if comparison_df is not None:
    save_analysis_session(comparison_df.to_dict('records'))

### 🎯 12. Exercício: Analise Seu Próprio Código

### CÉLULA PARA VOCÊ TESTAR COM UM PRÓPRIO CÓDIGO

### Instruções:
### 1. Cole abaixo um trecho do SEU código do ESPECIALISTA_EM_AI
### 2. Faça uma pergunta específica sobre ele
### 3. Execute as células para ver a análise

In [ ]:
# COLE SEU CÓDIGO AQUI
meu_codigo = """
# Cole aqui um trecho do seu repositório
# Exemplo: função do projeto de reconhecimento facial

def minha_funcao():
    # implementação...
    pass
"""

In [ ]:
# FAÇA SUA PERGUNTA AQUI
minha_pergunta = "Esta implementação está correta? Como poderia melhorar?"

In [ ]:
# EXECUTAR ANÁLISE
if meu_codigo.strip() and meu_codigo != '# Cole aqui um trecho do seu repositório\n# Exemplo: função do projeto de reconhecimento facial\n\ndef minha_funcao():\n    # implementação...\n    pass':
    print("🔍 Analisando seu código...\n")
    
    # Usar assistente inteligente
    if openai_client or ollama_ready:
        assistant = CodeAnalysisAssistant()
        
        print(f"📋 Seu código ({len(meu_codigo.split())} palavras):")
        print("="*50)
        print(meu_codigo[:500] + "..." if len(meu_codigo) > 500 else meu_codigo)
        print("="*50)
        print(f"\n❓ Sua pergunta: {minha_pergunta}")
        print("="*50)
        
        # Análise inteligente
        resultado = assistant.smart_analyze(meu_codigo, minha_pergunta)
        
        print("\n💡 ANÁLISE:")
        print("="*50)
        print(resultado)
        print("="*50)
        
        # Oferecer comparação se ambos estiverem disponíveis
        if assistant.use_openai and assistant.use_ollama:
            resposta = input("\n📊 Deseja comparar análises OpenAI vs Ollama? (s/n): ")
            if resposta.lower() == 's':
                assistant.compare_analyses(meu_codigo, minha_pergunta)
    else:
        print("⚠️  Configure pelo menos um modelo (OpenAI ou Ollama) primeiro.")
else:
    print("⚠️  Cole seu código na célula acima primeiro!")

### 📚 13. Conclusão e Próximos Passos

### 🎯 RESUMO DO NOTEBOOK

### ✅ O que aprendemos e configuramos:
1. **Configuração via .env**: Método seguro para armazenar chaves
2. **OpenAI API**: Configuração e uso para análise de código
3. **Ollama Local**: Instalação e uso de modelos locais gratuitos
4. **Análise Comparativa**: Cloud vs Local, performance, qualidade
5. **Sistema Inteligente**: Escolha automática do melhor modelo

### 🔧 Configuração atual:
- OpenAI: {'✅ Pronto' if openai_client else '❌ Não configurado'}
- Ollama: {'✅ Pronto' if ollama_ready else '❌ Não configurado'} ({OLLAMA_MODEL})
